In [ ]:
# ! pip install opencv-python

In [ ]:
# ! pip install path

In [ ]:
# ! pip install tensorflow_hub

In [ ]:
# ! pip install tqdm

In [ ]:
# ! pip install nibabel

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import cv2 as cv
from path import Path
import os 
import glob
import tensorflow_hub as hub
import os 
import pydicom as dicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import tensorflow as tf
from tensorflow import keras
from keras import layers
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from pydicom import dcmread
import nibabel as nib

In [ ]:
train_df = pd.read_csv("../input/rsna-2022-cervical-spine-fracture-detection/train.csv")

In [ ]:
train_df

In [ ]:
def load_dicom(path):
    img = dicom.dcmread(path)
    data = img.pixel_array
    data = data - np.min(data)
    if np.max(data) != 0 :
        data = data / np.max(data)
        
    
    data = (data*255).astype(np.uint8)
    
    return data

In [ ]:
train_dir = '../input/rsna-2022-cervical-spine-fracture-detection/train_images'
train_set = []
train_lab = []
train_idt = []
limit = 150

for i in tqdm(range(len(train_df))):
    idt = train_df.loc[i,'StudyInstanceUID']
    path = os.path.join(train_dir,idt)
    
    for im in os.listdir(path):
        
        dc = dicom.read_file(os.path.join(path,im))
        
        if dc.file_meta.TransferSyntaxUID.name == 'JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])':
            continue
            
        img = load_dicom(os.path.join(path,im))    
        img = cv.resize(img,(64,64))
        image = img_to_array(img)
        image = image/255.0
        
        train_set += [image]
        
        cur_lab = []
        cur_lab.append(train_df.loc[i,'patient_overall'])
        cur_lab.append(train_df.loc[i,'C1'])
        cur_lab.append(train_df.loc[i,'C2'])
        cur_lab.append(train_df.loc[i,'C3'])
        cur_lab.append(train_df.loc[i,'C4'])
        cur_lab.append(train_df.loc[i,'C5'])
        cur_lab.append(train_df.loc[i,'C6'])
        cur_lab.append(train_df.loc[i,'C7'])
        
        train_lab += [cur_lab]
    train_idt += [idt]
        
    i = i+1
        
    if i == limit:
        break
        

In [ ]:
X_train = np.array(train_set)
Y_train = np.array(train_lab)


In [ ]:
X_train.shape

In [ ]:
Y_train.shape

In [ ]:
test_df = ['1.2.826.0.1.3680043.22327', '1.2.826.0.1.3680043.25399', '1.2.826.0.1.3680043.5876']

test_dir = '../input/rsna-2022-cervical-spine-fracture-detection/test_images'
test_set = []
test_idt = []


for i in tqdm(range(len(test_df))):
    idt = test_df[i]
    path = os.path.join(test_dir, idt)
    
    for im in os.listdir(path):
        
        dc = dicom.read_file(os.path.join(path,im))
        
        if dc.file_meta.TransferSyntaxUID.name == 'JPEG Lossless, Non-Hierarchical, First-Order Prediction (Process 14 [Selection Value 1])':
            continue
            
        img = load_dicom(os.path.join(path,im))    
        img = cv.resize(img,(64,64))
        image = img_to_array(img)
        image = image/255.0
        
        test_set += [image]
        
    test_idt += [idt]
        
    
        

        

In [ ]:
X_test = np.array(test_set)

In [ ]:
test_idt


In [ ]:
X_test.shape

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(filters = 64, kernel_size = (4,4), input_shape = (64,64,1), activation = 'relu', kernel_initializer = 'he_normal'))
model.add(keras.layers.MaxPooling2D(pool_size = (2,2)))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(filters = 64, kernel_size = (4,4), activation = 'relu', kernel_initializer = 'he_normal'))
model.add(keras.layers.MaxPooling2D(pool_size = (2,2)))
model.add(keras.layers.Dropout(0.20))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Conv2D(filters = 64, kernel_size = (4,4), activation = 'relu', kernel_initializer = 'he_normal'))
model.add(keras.layers.MaxPooling2D(pool_size = (2,2)))
model.add(keras.layers.Dropout(0.25))
model.add(keras.layers.BatchNormalization())
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(100, activation = 'relu', kernel_initializer = 'he_normal'))
model.add(keras.layers.Dense(8, activation = 'softmax'))


In [ ]:
model.compile(loss = 'binary_crossentropy', metrics = ['accuracy'], optimizer = 'RMSprop' )
callback = keras.callbacks.EarlyStopping(monitor = 'loss', patience = 8)


In [ ]:
hist = model.fit(X_train, Y_train, epochs = 100, batch_size = 64, verbose = 1, callbacks = [callback])

In [ ]:
y_pred = model.predict(X_test)
res = pd.DataFrame(columns = train_df.columns, index = range(len(test_idt)))
y_mean = np.mean(y_pred)

In [ ]:
for i in tqdm(range(len(test_idt))):
    res.loc[i,'StudyInstanceUID'] = test_idt[i]
    rows = np.int64(y_pred[i] > y_mean)
    res.loc[i, 'patient_overall'] = rows[0]
    res.loc[i, 'C1'] = rows[1]
    res.loc[i, 'C2'] = rows[2]
    res.loc[i, 'C3'] = rows[3]
    res.loc[i, 'C4'] = rows[4]
    res.loc[i, 'C5'] = rows[5]
    res.loc[i, 'C6'] = rows[6]
    res.loc[i, 'C7'] = rows[7]

In [ ]:
res

In [ ]:
means = res[['patient_overall', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']].mean().to_dict()

In [ ]:
means

In [ ]:
test_df2 = pd.read_csv('../input/rsna-2022-cervical-spine-fracture-detection/test.csv')
test_df2['fractured'] = test_df2['prediction_type'].map(means)

In [ ]:
test_df2[['row_id','fractured']].to_csv('submission.csv', index = False, float_format = '%.1g')